In [1]:
!wget -O video.mp4 https://github.com/hassanTV/AI-intro/raw/main/IoT.mp4

import cv2
import numpy as np
import matplotlib.pyplot as plt

cap = cv2.VideoCapture('IoT.mp4')
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)) # Just to learn it
frame_height =int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

ret, frame1 = cap.read()
ret, frame2 = cap.read()
i = 1
while ret:
    # Get absolute difference image, dilate it, convert it to binary and dilate it for better contours calculation
    diff = cv2.absdiff(frame1, frame2) # abs(frame1 - frame2)
    gray = cv2.cvtColor(diff, cv2.COLOR_BGR2GRAY)
    blur = cv2.GaussianBlur(gray, (5,5), 0)
    _, thresh = cv2.threshold(blur, 20, 255, cv2.THRESH_BINARY)
    dilated = cv2.dilate(thresh, None, iterations=3)
    # CHAIN_APPROX_NONE stores every pixel, and CHAIN_APPROX_SIMPLE stores only endpoints of the lines that forms the contour
    # RETR_TREE won't matter because it's how the retrieved contours are ordered (hierarchy)
    contours, _ = cv2.findContours(dilated, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE) # Boundaries of connected components

    # Draw Image, difference image ()Contours
    """plt.figure()
    plt.imshow(frame1)
    plt.figure()
    plt.imshow(frame2)
    plt.figure()
    plt.imshow(diff)
    plt.figure()
    plt.imshow(dilated)
    plt.figure()
    cv2.drawContours(frame1, contours, -1, (0, 255, 0), 3)  
    plt.imshow(frame1)
    break"""

    # Mark large contours as moving object
    for contour in contours:
        (x, y, w, h) = cv2.boundingRect(contour)
        if cv2.contourArea(contour) < 900: # Safer to increase the expected moving object area to reduce false positives
            continue
        cv2.rectangle(frame1, (x, y), (x+w, y+h), (0, 255, 0), 2)
        cv2.putText(frame1, "Status: Movement", (10, 20), cv2.FONT_HERSHEY_SIMPLEX, 3, (0, 0, 255), 3)

    # Save result image
    image = cv2.resize(frame1, (1280,720))
    cv2.imwrite("Result%04d.jpg" % i, image)
    i += 1

    # Read next frame
    frame1 = frame2
    ret, frame2 = cap.read()

**Save & download results video**

In [2]:
cap.release()
# Create the resulted video file, then download it manually if needed and watch the result
!ffmpeg -framerate 30 -r 30 -i 'Result%04d.jpg' -vcodec mpeg4 -y result.mp4 # Save video
from google.colab import files
files.download('result.mp4')

'ffmpeg' is not recognized as an internal or external command,
operable program or batch file.


ModuleNotFoundError: No module named 'google'

**Exercise: Run and tune the code above to detect motion on any other video, this can be an example: https://github.com/heshameraqi/Image_Processing_Computer_Vision_1/raw/main/motion_detection_car_overhead.avi**